# 读数据

In [1]:
import pywxdump

wx_data=pywxdump.get_wx_info()[0]
key=wx_data['key']
wx_dir=wx_data['wx_dir']
yourself=wx_data['wxid']
# pywxdump.batch_decrypt(key,wx_dir+'\\Msg','./data',True)

[-] WeChat Current Version Is Not Supported(not get account,mobile,nickname,mail)


# 洗数据

In [2]:
import sqlite3
import pandas as pd
import time

# 连接到数据库
conn = sqlite3.connect('./app/DataBase/Msg/MicroMsg.db')
cursor = conn.cursor()
name = pd.read_sql("SELECT userName, nickName, remark FROM Contact", conn)
# 关闭连接
conn.close()
name.dropna(inplace=True)
name=name[name['Remark']!='']
username2remark= dict(zip(name['UserName'], name['Remark']))
username2remark.keys


def timestamp2time(timestamp):
    local_time = time.localtime(timestamp)  # 转换为本地时间结构
    formatted_time = time.strftime("%Y-%m-%d %H:%M:%S", local_time)
    return formatted_time


def show_table_with_pandas(db_path, table_name):
    conn = sqlite3.connect(db_path)
    
    try:
        # 使用 pandas 读取整个表
        df = pd.read_sql(f"SELECT * FROM {table_name};", conn)
        
        # 显示数据
        return df
        
    except sqlite3.Error as e:
        print(f"查询表数据时出错: {e}")
    finally:
        conn.close()

def id2name(id):
    return username2remark[id] if id in username2remark.keys() else id


def feeds_with_comments(feeds_df, comments_df):
    comments_df['FromUserName'] = comments_df['FromUserName'].apply(id2name)
    # comments_df = comments_df[comments_df['FromUserName'] != 'unknown']
    feeds_df['UserName']= feeds_df['UserName'].apply(id2name)
    # feeds_df = feeds_df[feeds_df['UserName'] != 'unknown']
    feed_comments = []

    for index, row in feeds_df.iterrows():
        feedId=row['FeedId']
        if feedId in comments_df['FeedId'].values:
            comments= (comments_df[comments_df['FeedId'] == feedId]['FromUserName']).to_list()
            feed_comments.append([row['UserName'],comments])
    return pd.DataFrame(feed_comments, columns=['UserName', 'Comments'])

In [3]:
feeds_df=show_table_with_pandas('./app/DataBase/Msg/Sns.db', 'FeedsV20')
comments_df=show_table_with_pandas('./app/DataBase/Msg/Sns.db', 'CommentV20')
feeds2comments=feeds_with_comments(feeds_df, comments_df)
feeds2comments

,UserName,Comments
0,陈佳怡,"[李梓棋, 陈佳怡, 李梓棋]"
1,吕博研,[王飒]
2,流统伏梦婷,"[欧阳志欣, 流统伏梦婷, 欧阳志欣, 流统白锦玲, 流统李佳潞, 邹明君专硕1808920..."
3,流统伏梦婷,"[欧阳志欣, 流统伏梦婷, 邹明君专硕18089204572, 流统伏梦婷, 邹明君专硕18..."
4,王慧慧,"[王慧慧, 谢晓玉, 张婷]"
...,...,...
412,林德奕,"[冯华健, 王烨云, 张楚雄]"
413,陈栩,"[刘耘恺, 苑汝堂, 陈栩, 刘耘恺, 石梁梨, 李健冬, wxid_m6yq15lb7hs..."
414,孟晓宇,"[孟晓宇, 孟晓宇, 孟晓宇]"
415,张迦南,"[孟泽宇, 王春苗, 张迦南, 张迦南, 万欣怡, 申弘烨, 孟泽宇, 何艳飞, 尹怡然, ..."


In [4]:
import networkx as nx
import pandas as pd
import os

# 假设你的数据是这样的
data = feeds2comments
df = pd.DataFrame(data)

# 创建有向图
if os.path.exists("graph.gexf"):
    G = nx.read_gexf("graph.gexf")  # 读取
else:
    G = nx.Graph()
# 添加节点和边
for _, row in df.iterrows():
    poster = row['UserName']
    commenters = row['Comments']
    
    # 确保发帖人作为节点存在
    G.add_node(poster)
    
    # 添加从评论者到发帖人的边
    for commenter in commenters:
        G.add_node(commenter)
        G.add_edge(commenter, poster)
G.remove_node(yourself)
nx.write_gexf(G, "graph.gexf")  # 保存

In [5]:
from pyvis.network import Network

net = Network(notebook=True, directed=False, height="750px", width="100%")
net.from_nx(G)
net.show_buttons(filter_=['physics'])  # 显示控制按钮
net.show("comment_network.html")  # 生成HTML文件

comment_network.html
